In [3]:
import random
from collections import deque
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout
import logging

from gx_utils import *


logging.basicConfig(format="%(message)s", level=logging.INFO)

In [4]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [5]:
def tree_search(lists, goal_test, strategy="bf", bound=False):
    frontier = deque()

    frontier.append(([], lists))
    n=0
    while frontier:
        n+=1
        
        if strategy == "bf":
            state = frontier.popleft()
        elif strategy == "df":
            state = frontier.pop()
        
        selected,available=state
        
        if goal_test(selected):
            logging.info(f"Found a solution in {n:,} steps: {selected}")
            #print(f"Found a solution in {n:,} steps: {selected}")
            solution=selected
            break
        
        for i,newlist in enumerate(available):
            
            newState=(selected+[newlist],available[i+1 :])
            frontier.append(newState)
        
    return solution

In [12]:
from queue import PriorityQueue

def tree_search2(lists, goal_test,slzCost, priority_function,states):
    frontier = PriorityQueue()

    #frontier.push(([], lists))
    state=((), lists) #initial state
    solution=lists
    n=0
    while state is not None:
        n+=1
        
        selected,available=state
        
        if goal_test(selected):
            logging.info(f"Found a solution in {n:,} steps: {selected}")
            
            if  slzCost(selected)<slzCost(solution):
                solution=selected
            break
        
        for i,newlist in enumerate(available):
            if not set(newlist) < set([item for sublist in selected for item in sublist]):
                
                newState=(selected+(newlist,),available[i+1 :])
                states[newState]=slzCost(selected)
                
                frontier.put((priority_function(newState),newState))
        
        if frontier:
            state = frontier.get()[1]
        else:
            state = None
        
    return solution

In [16]:
def goal_test_gen(N):
    def goal_test(slz):
        #print(slz)
        if set([item for sublist in slz for item in sublist])==set(range(N)):
            return True
        else:
            return False
    return goal_test

def slzCost(solution):
    return (sum(len(_) for _ in solution)-N)/N
    #return len([item for sublist in state[0] for item in sublist])

def priority_function(newState):
    selected=newState[0][:-1]
    newlist=newState[0][-1]
    return len(newlist)-3*len(set(newlist)-set([item for sublist in selected for item in sublist]))
    

for N in [5, 10, 20]:
    lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    filteredLists=sorted(list(list(_) for _ in set(tuple(l) for l in lists)), key=lambda l:len(l)) #to better see the problem
    
    #solution=tree_search(lists,goal_test_gen(N)) #breadth first search
    
    states=dict()

    tuples=tuple(tuple(sublist) for sublist in lists)
    #solution=tree_search2(tuples,goal_test_gen(N),slzCost,lambda l:len(states),states) #second breadth first
    #print(f"Solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)")
    
    solution2=tree_search2(tuples,goal_test_gen(N),slzCost,lambda s: priority_function(s)+states[s],states)
    print(f"Solution for N={N}: w={sum(len(_) for _ in solution2)} (bloat={(sum(len(_) for _ in solution2)-N)/N*100:.0f}%)")
    
    #solution2=tree_search2(tuples,goal_test_gen(N),slzCost,lambda s: priority_function(s),states)
    #print(f"Greedy Solution for N={N}: w={sum(len(_) for _ in solution2)} (bloat={(sum(len(_) for _ in solution2)-N)/N*100:.0f}%)")


Found a solution in 23 steps: ((0,), (1, 3), (2, 4))
Found a solution in 102 steps: ((1, 3, 6), (8, 2, 7), (0, 9, 4, 5))
Found a solution in 446 steps: ((16, 9, 19, 6), (0, 1, 2, 7), (4, 7, 11, 12, 15, 16, 18), (0, 3, 5, 8, 9, 10, 13, 14, 17))


Solution for N=5: w=5 (bloat=0%)
Solution for N=10: w=10 (bloat=0%)
Solution for N=20: w=24 (bloat=20%)
